In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[5],3c
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,5e-25
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[5]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -7.510005e-06                    1 -1.189472e-05
0.000750        2 -6.919919e-06                    2 -1.193219e-05
0.001052        3 -5.672550e-06                    3 -1.116911e-05
0.001476        4 -4.345040e-06                    4 -1.073718e-05
0.002070        5 -2.961614e-06                    5 -1.026452e-05
0.002904        6 -1.508665e-06                    6 -9.788158e-06
0.004074        7  1.312034e-08                    7 -9.265546e-06
0.005714        8  1.605370e-06                    8 -8.722281e-06
0.008015        9  3.261757e-06                    9 -8.158414e-06
0.011243       10  4.993077e-06                   10 -7.569195e-06
0.015771       11  6.800780e-06                   11 -6.981155e-06
0.022122       12  8.687378e-06                   12 -6.343284e-06
0.031031       13  1.066664e-05                   13 -5.686093e-06
0.043528       14  1.330962e-05                   14 -4.812906e-06
0.061057       15  2.370610e-05                   15 -1.183840e-06
0.085645       16  5.040278e-05                   16  8.392172e-06
0.120136       17  9.475622e-05                   17  2.460375e-05
0.168516       18  1.584078e-04                   18  4.805160e-05
0.236378       19  2.476754e-04                   19  8.101896e-05
0.331549       20  3.710050e-04                   20  1.264901e-04
0.465100       21  5.390742e-04                   21  1.899026e-04
0.652400       22  7.402300e-04                   22  2.704952e-04
0.915100       23  8.189034e-04                   23  3.038921e-04
1.283650       24  7.399414e-04                   24  2.750342e-04
1.800600       25  6.138147e-04                   25  2.275111e-04
2.525700       26  4.694976e-04                   26  1.727930e-04
3.542800       27  3.464878e-04                   27  1.259826e-04
4.969550       28  2.503132e-04                   28  8.939504e-05
6.970850       29  1.758310e-04                   29  6.083137e-05
9.778100       30  1.186502e-04                   30  3.866706e-05
13.715850      31  7.520215e-05                   31  2.154251e-05
19.239350      32  4.261063e-05                   32  8.354893e-06
26.987250      33  1.836887e-05                   33 -1.784231e-06
37.855300      34  1.454620e-07                   34 -9.676811e-06
53.100050      35 -1.608919e-05                   35 -1.734487e-05
73.887500      36 -2.683766e-05                   36 -2.338910e-05
97.662500      37 -3.027027e-05                   37 -2.595925e-05
121.437500     38 -3.000015e-05                   38 -2.677433e-05
145.212500     39 -2.554705e-05                   39 -2.543427e-05
168.987500     40 -1.755069e-05                   40 -2.294166e-05
192.762500     41 -8.247866e-07                   41 -2.045047e-05
216.537500     42  7.633501e-05                   42 -7.179163e-07
240.312500     43  3.443560e-04                   43  1.260550e-04
264.087500     44  1.010529e-03                   44  5.179012e-04
287.862500     45  2.350140e-03                   45  1.430196e-03
311.637500     46  4.644299e-03                   46  3.208865e-03
335.412500     47  8.088632e-03                   47  6.168141e-03
359.187500     48  1.256919e-02                   48  1.043469e-02
382.962500     49  1.710819e-02                   49  1.517025e-02
406.737500     50  2.238785e-02                   50  2.065659e-02
430.512500     51  2.837063e-02                   51  2.659098e-02
454.287500     52  3.504004e-02                   52  3.294187e-02
478.062500     53  4.241032e-02                   53  3.979456e-02
501.837500     54  5.043117e-02                   54  4.731695e-02
525.612500     55  5.922360e-02                   55  5.583978e-02
549.387500     56  6.874208e-02                   56  6.568662e-02
573.162500     57  7.903999e-02                   57  7.67254

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -32.497554  0.000000e+00 -32.497554 -32.547107   
0.000624    2     -32.497554  3.434516e-09 -32.497554 -32.547107   
0.000876    3     -32.497554  4.838863e-09 -32.497554 -32.547107   
0.001229    4     -32.497554  6.869621e-09 -32.497554 -32.547107   
0.001723    5     -32.497554  9.808422e-09 -32.497554 -32.547107   
0.002417    6     -32.497554  1.406033e-08 -32.497554 -32.547107   
0.003391    7     -32.497554  2.021224e-08 -32.497554 -32.547107   
0.004757    8     -32.497554  2.911249e-08 -32.497554 -32.547107   
0.006672    9     -32.497554  4.198726e-08 -32.497554 -32.547107   
0.009359    10    -32.497554  6.061046e-08 -32.497554 -32.547107   
0.013128    11    -32.497554  8.754644e-08 -32.497554 -32.547107   
0.018415    12    -32.497554  1.265036e-07 -32.497554 -32.547107   
0.025830    13    -32.497554  1.828429e-07 -32.497554 -32.547108   
0.036232    14    -32.497554  2.643238e-07 -32.497554 -32.547108   
0.050823    15    -32.497554  3.831480e-07 -32.497554 -32.547108   
0.071291    16    -32.497554  5.739773e-07 -32.497554 -32.547108   
0.100000    17    -32.497554  9.227093e-07 -32.497554 -32.547108   
0.140271    18    -32.497555  1.585707e-06 -32.497553 -32.547108   
0.196760    19    -32.497555  2.841233e-06 -32.497552 -32.547108   
0.275997    20    -32.497555  5.206994e-06 -32.497550 -32.547108   
0.387100    21    -32.497555  9.641159e-06 -32.497545 -32.547109   
0.543100    22    -32.497553  1.792166e-05 -32.497535 -32.547108   
0.761700    23    -32.497549  3.285419e-05 -32.497516 -32.547107   
1.068500    24    -32.497542  5.557531e-05 -32.497486 -32.547106   
1.498800    25    -32.497533  8.483607e-05 -32.497448 -32.547104   
2.102400    26    -32.497524  1.200332e-04 -32.497404 -32.547103   
2.949000    27    -32.497517  1.598983e-04 -32.497357 -32.547103   
4.136600    28    -32.497513  2.041996e-04 -32.497309 -32.547105   
5.802500    29    -32.497512  2.532688e-04 -32.497259 -32.547109   
8.139200    30    -32.497518  3.074786e-04 -32.497211 -32.547118   
11.417000   31    -32.497532  3.672632e-04 -32.497164 -32.547131   
16.014700   32    -32.497557  4.331255e-04 -32.497123 -32.547152   
22.464000   33    -32.497597  5.057072e-04 -32.497091 -32.547183   
31.510500   34    -32.497657  5.856619e-04 -32.497072 -32.547228   
44.200100   35    -32.497744  6.730428e-04 -32.497072 -32.547291   
62.000000   36    -32.497865  7.598873e-04 -32.497105 -32.547379   
85.775000   37    -32.498016  8.346116e-04 -32.497181 -32.547493   
109.550000  38    -32.498158  8.923445e-04 -32.497266 -32.547607   
133.325000  39    -32.498314  9.631050e-04 -32.497350 -32.547732   
157.100000  40    -32.498497  1.074809e-03 -32.497422 -32.547880   
180.875000  41    -32.498728  1.256438e-03 -32.497472 -32.548067   
204.650000  42    -32.499105  1.630157e-03 -32.497475 -32.548379   
228.425000  43    -32.500027  2.767650e-03 -32.497260 -32.549168   
252.200000  44    -32.501968  5.678193e-03 -32.496290 -32.550878   
275.975000  45    -32.505592  1.214864e-02 -32.493444 -32.554143   
299.750000  46    -32.511752  2.492692e-02 -32.486825 -32.559796   
323.525000  47    -32.521457  4.771179e-02 -32.473745 -32.568847   
347.300000  48    -32.535850  8.488533e-02 -32.450964 -32.582381   
371.075000  49    -32.555922  1.403570e-01 -32.415566 -32.601395   
394.850000  50    -32.581233  2.138498e-01 -32.367384 -32.625535   
418.625000  51    -32.612387  3.080545e-01 -32.304332 -32.655378   
442.400000  52    -32.649931  4.254985e-01 -32.224432 -32.691401   
466.175000  53    -32.694344  5.685959e-01 -32.125748 -32.734148   
489.950000  54    -32.746042  7.397358e-01 -32.006307 -32.784067   
513.725000  55    -32.805258  9.409807e-01 -31.864277 -32.841472   
537.500000  56    -32.872203  1.174720e+00 -31.697484 -32.906381   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')